# Test model on sources it was not trained on

In [ ]:
import os
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
import numpy as np
import pandas as pd
import transformers
from sklearn.metrics import precision_score, recall_score, f1_score, mean_squared_error
from apex import amp

In [ ]:
# set path to media-bias-prediction repository 
repo_path = os.path.dirname(os.getcwd())
os.chdir(os.path.join(repo_path, 'other_notebooks'))

## Loading and preparing data

In [ ]:
os.chdir(os.path.join(repo_path, 'data_preparation','allsides_data'))

bias_test = torch.load('allsides_bias_test.pt')
text_test = torch.load('allsides_contents_text_test.pt')
mask_test = torch.load('allsides_contents_mask_test.pt')

os.chdir(os.path.join(repo_path, 'other_notebooks'))

In [ ]:
os.chdir(os.path.join(repo_path, 'data_preparation','allsides_data'))

allsides_source_test = np.load('allsides_source_test.npy', allow_pickle=True).flatten()

# sources to be removed:
wrongly_labeled = ['RightWingWatch']
news_aggregators = ['Drudge Report', 'Real Clear Politics', 'Yahoo News', 'RightWingWatch'] 
tabloids = ['New York Daily News', 'Daily Mail', 'New York Post'] 
unwanted_sources = wrongly_labeled + news_aggregators + tabloids
# creating boolean array to mark unwanted sources
boolean_array_test = np.full((len(allsides_source_test), ), False)

for source in unwanted_sources:
    boolean_array_test += allsides_source_test==source 
# boolean to remove aggregators
inverted_boolean_array_test = np.invert(boolean_array_test)

# bias
bias_test = bias_test[inverted_boolean_array_test]

# text and masks
text_test = text_test[inverted_boolean_array_test]
mask_test = mask_test[inverted_boolean_array_test]

# sources
allsides_source_test = allsides_source_test[inverted_boolean_array_test]

os.chdir(os.path.join(repo_path, 'other_notebooks'))

In [ ]:
test_set = TensorDataset(text_test, mask_test, bias_test)

## Model class

In [ ]:
class Model(nn.Module):
    def __init__(self, hidden_size, num_labels, droput_prob, bert_model_module, output_attentions=False, pooled_output = True):
        super().__init__()
        self.hidden_size = hidden_size
        self.num_labels = num_labels
        self.output_attentions = output_attentions
        self.pooled_output = pooled_output

        self.bert = bert_model_module
        self.dropout = nn.Dropout(dropout_prob)
        self.linear = nn.Linear(hidden_size,hidden_size)
        self.tanh = nn.Tanh()

        self.classifier_layer = nn.Linear(hidden_size, num_labels)
        
    def forward(self, text, mask):
        # token_type_ids and position_ids are created automaticly 
        bert_out = self.bert(input_ids = text, attention_mask = mask)
        
        if self.pooled_output:
            # Choosing only CLS token output and apply linear layer + TanH 
            pooled_out = bert_out[1]
            # Applying dropout
            pooled_out = self.dropout(pooled_out)

            out = self.classifier_layer(pooled_out)
        else:
            ### Last Layer average
            # summing up over sequence lenght and devide by unmasked sequence length 
            # resulting in tensor with shape (batch_size,hidden_size)
            last_layer = torch.sum(bert_out[0], dim=1)/torch.sum(mask,dim=1).reshape([len(mask),1])
            last_layer = self.tanh(self.linear(last_layer))
            last_layer = self.dropout(last_layer)
            out = self.classifier_layer(last_layer)
               
        # Saving attention layer outputs if set True
        if self.output_attentions:
            out = out, bert_out[2]
        
        return out

## Preparing model

In [ ]:
##### Loading Bert 
BertModel = transformers.BertModel

### Device to run model on, either GPU or CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

### Model inputs
hidden_size = 768
num_labels = 5 
dropout_prob = 0.1

mixed_precision = True

### Hyperparameters
batch_size = 16 
learning_rate = 2e-5

##### Initilize and configure Bert
bert_model = BertModel.from_pretrained('bert-base-uncased') 

##### Initilize model 
model = Model(hidden_size, num_labels, dropout_prob, bert_model, pooled_output=True).to(device)
### Applying mixed precision to speed up model inference 
if mixed_precision:
    model = amp.initialize(model, opt_level="O1") 

## Prediction function

In [ ]:
def prediction_fct():
    model.eval()
    start_time = time.time()

    data = DataLoader(test_set, batch_size=batch_size)
    test_loss = 0
    test_predicted_values = []

    number_of_batches = len(test_set)//batch_size
    update = 500
    batch_counter = 0

    for text, mask, label in data:
        text, mask, label = text.to(device), mask.to(device), label.to(device)

        with torch.no_grad():
            output = model(text,mask)
            test_predicted_values.append(output.argmax(1))

        batch_counter += 1
        if (batch_counter % update == 0) or (batch_counter == 100):
            update_time = time.time() - start_time
            minutes = int(update_time // 60)
            seconds = round(update_time % 60)
            print(f'{batch_counter:5} of {number_of_batches} batches done after {minutes:2} min {seconds:2} sec ')
            print('-----------------------------------------------')
    
    print('all batches done')
    print('-----------------------------------------------')
    test_predicted_values = torch.cat(test_predicted_values).cpu().numpy()
    test_true_values = bias_test.numpy()
    
    return test_predicted_values, test_true_values

## Creating results

In [ ]:
small_group = ['Daily Kos','Washington Monthly', 'FiveThirtyEight',                
               'The Washington Examiner', 'FrontPage Magazine']                    
large_group = ['Politicus USA', 'ABC News', 'Reuters', 'Fox News', 'CNS News']

for group_type in ['small', 'large']:
    for model_weights in ['with_sources', 'without_sources']:
        for run in range(1,4):
            ##### Initilize and configure Bert
            bert_model = BertModel.from_pretrained('bert-base-uncased') 
            ##### Initilize model 
            model = Model(hidden_size, num_labels, dropout_prob, bert_model, pooled_output=True).to(device)
            ### Applying mixed precision to speed up model inference 
            if mixed_precision:
                model = amp.initialize(model, opt_level="O1", verbosity=0)

            ### Load model weights
            if model_weights=='with_sources':
                checkpoint = torch.load(os.path.join(repo_path, 'deep_learning_models', 'weights', f'amp_checkpoint_allsides_aggregators_tabloids_duplicates_removed_rerun_{run}_epoch3.pt')) 
            else:
                checkpoint = torch.load(os.path.join(repo_path, 'deep_learning_models', 'weights', f'amp_checkpoint_allsides_robustness_check_{group_type}_rerun_{run}_epoch3.pt')) 

            model.load_state_dict(checkpoint['model'])
            
            ### make predictions
            print('+++++++++++++++++++++++++++++++++++++++++++++++') 
            print(f' predicting on {group_type} group {model_weights} run {run}')
            print('+++++++++++++++++++++++++++++++++++++++++++++++')
                   
            test_predicted_values, test_true_values = prediction_fct()
            
            # scores of removed sources
            if group_type =='small':
                removed_sources = small_group
            else:
                removed_sources = large_group
                
            removed_sources_scores = []
            for i,source in enumerate(removed_sources):
                pred = test_predicted_values[allsides_source_test==source]
                true = test_true_values[allsides_source_test==source]

                removed_sources_accuracy = (pred==true).sum()/len(pred)

                removed_sources_scores.append([removed_sources_accuracy, len(pred)])

            # scores of remaining sources
            kept_sources_boolean = (allsides_source_test!=removed_sources[0])&(allsides_source_test!=removed_sources[1])&(allsides_source_test!=removed_sources[2])&(allsides_source_test!=removed_sources[3])&(allsides_source_test!=removed_sources[4])

            kept_sources_pred = test_predicted_values[kept_sources_boolean]
            kept_sources_true = test_true_values[kept_sources_boolean]

            kept_sources_accuracy = (kept_sources_pred==kept_sources_true).sum()/len(kept_sources_pred)

            removed_sources_scores.append([kept_sources_accuracy, len(kept_sources_pred)])

            removed_sources_scores = pd.DataFrame(np.array(removed_sources_scores).transpose() , columns=removed_sources+['remaining'], index=['Acc', 'Frequency' ]).round(4)
            print(removed_sources_scores)
            removed_sources_scores.to_csv(f'scores/accuracy_scores_{group_type}_{model_weights}_run_{run}.csv', index=False)
            